# GEOGRAPHIC PARTS
# IMPORT LIBS

In [ ]:
import pandas as pd
import plotly.express as px
import math
import numpy as np
import geopy.distance

# OPEN DATASET

In [ ]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_00_dropna.csv')
df.head()

# DELETE ALL LATITUDE OUT OF RANGE

In [ ]:
df = df.loc[(df['pickup_latitude'] >= 40 ) & (df['pickup_latitude'] <= 42 ) & (df['dropoff_latitude'] >= 40 ) & (df['dropoff_latitude'] <= 42 ) & (df['pickup_longitude'] >= -76 ) & (df['pickup_longitude'] <= -72 ) & (df['dropoff_longitude'] >= -76 ) & (df['dropoff_longitude'] <= -72 )]
df.head()

# CALCUL DISTANCE WITH GEOPY LIB

In [ ]:
df['Distance (miles)'] = df.apply(lambda row: geopy.distance.geodesic((row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])).miles, axis=1)

df.head()

On this plot, we can see too many drive with high distance but cheap price AND too many drive with cheap distance but high price.
So, we gonna limit this by adding a price by kmfeature and try to drpo rows with price by km useless

# VISUALISATION OF OUTLIERS
## DISPLAY DISTANCE DISTRIBUTION

In [ ]:
# fig = px.violin(df, y="Distance (miles)", box=True,
#                 # points="all",
#                 # hover_data=df.columns,
#                 title='Violin plot of Distance distribution'
#                 )
# fig.show()

# Calcul D’Outliers :
X < q1 - 1.5 x iqr X > q3 - 1.5 x iqr

In [ ]:
q1 = df['Distance (miles)'].quantile([.25])
q1 = q1[0.25]
q3 = df['Distance (miles)'].quantile([.75])
q3 = q3[0.75]

iqr = q3 - q1

In [ ]:
print(f" IQR = {iqr}, q1 = {q1}, q3 = {q3}")

In [ ]:
outliers_superieur = q3 + (1.5 * iqr)
outliers_inferieur = q1 - (1.5 * iqr)

print(f'Outliers Inférieur = {outliers_inferieur}')
print(f'Outliers Supérieur = {outliers_superieur}')

# DROP OUTLIERS

In [ ]:
df = df.loc[(df['Distance (miles)'] <= outliers_superieur) & (df['Distance (miles)'] >= outliers_inferieur)]
df.shape

# DISPLAY NEW VIOLIN PLOT WITHOUT OUTLIERS

In [ ]:
# fig = px.violin(df, y="Distance (miles)", box=True,
#                 # points="all",
#                 # hover_data=df.columns,
#                 title='Violin plot of Distance distribution'
#                 )
# fig.show()

# SCATTER PLOT OF FARE AMOUNT BY DISTANCE

In [ ]:
# fig = px.scatter(df, x='Distance (miles)', y='fare_amount', color='passenger_count')
# fig.show()

# DISPLAY NUMBER OF ROWS AFTER PREPROCESSING

In [ ]:
df.shape

# EXPORT CSV

In [ ]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv', index=False)